In [1]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [2]:
%%capture pwd
!pwd

If running this workbook on `Google Colab`:
- Uncomment line and set `uid` equal to the output of the previous cell
- Set `LOCAL` to `False` 

In [3]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
# uid='<PASTE UID FROM BELOW HERE>'
print(uid)

2021-05-17-why-should-we-care-about-obesity


In [4]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [5]:
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
height, width

(300, 500)

# Fig 1

In [6]:
raw_file_path = "raw/Figures.xlsx"
if not LOCAL:
    raw_file_path = eco_git_path.split("/data")[0] + "/" + raw_file_path
df = (
    pd.read_excel(
        raw_file_path,
        skiprows=1,
        nrows=3,
    )
    .dropna(how="all", axis=1)
    .dropna(how="all", axis=0)
)
df = (
    df.set_index("Unnamed: 0")
    .stack()
    .reset_index()
    .set_index(["level_1", "Unnamed: 0"])
    .unstack()[0]
    .reset_index()
)
df.columns = ["year", "morbidly", "obese"]
df["all"] = df["morbidly"] + df["obese"]
df["year"] = df["year"].astype(str)

In [7]:
f = "fig1_obesity"
f1 = eco_git_path + f + ".csv"
if LOCAL:
    df.to_csv("data/" + f + ".csv")
    f += local_suffix
    open("visualisation/" + f + ".html", "w").write(
        vega_embed.replace(
            "JSON_PATH",
            (eco_git_path + f + ".csv")
            .replace("/data/", "/visualisation/")
            .replace(".csv", ".json"),
        )
    )
    f1 = df
df.head()

,year,morbidly,obese,all
0,1993,0.824392,14.904475,15.728867
1,1994,1.015056,15.668642,16.683698
2,1995,0.865891,16.444999,17.310889
3,1996,0.916274,17.488879,18.405152
4,1997,1.599698,18.428014,20.027711


In [8]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "year:T",
        axis=alt.Axis(
            grid=False,
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
        ),
    )
)
area1 = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.8,
    stroke=colors["eco-gray"],
    strokeWidth=0.5,
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-turquiose"], offset=0.9),
        ],
        x1=0.8,
        x2=1,
        y1=1,
        y2=0,
    ),
).encode(
    y=alt.Y(
        "obese:Q",
        axis=alt.Axis(
            grid=True,
            title="%",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            labelAlign="left",
            ticks=False,
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=20,
            titleY=37,
            titleAngle=0,
            titleFontSize=10,
            titleFontWeight="normal",
            titleAlign="left",
            tickCount=4,
            format=".0f",
        ),
    )
)
area2 = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.8,
    stroke=colors["eco-gray"],
    strokeWidth=0.5,
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-blue"], offset=0.9),
        ],
        x1=0.8,
        x2=0.8,
        y1=0.8,
        y2=0,
    ),
).encode(y=alt.Y("all:Q"))
label1 = (
    alt.Chart(
        pd.DataFrame(
            [{"x": "1999-03-15", "y": 24, "t": "Obese", "c": colors["eco-turquiose"]}]
        )
    )
    .mark_text(yOffset=5, size=10, align="right")
    .encode(text="t", x="x:T", y="y:Q", color=alt.Color("c:N", scale=None))
)
label2 = (
    alt.Chart(
        pd.DataFrame(
            [
                {
                    "x": "2004-03-01",
                    "y": 26,
                    "t": "Morbidly obese",
                    "c": colors["eco-blue"],
                }
            ]
        )
    )
    .mark_text(yOffset=-5, size=10, align="right")
    .encode(text="t", x="x:T", y="y:Q", color=alt.Color("c:N", scale=None))
)
layer = (
    (area2 + area1 + label1 + label2)
    .configure_view(stroke=None)
    .properties(title="")
    .properties(height=300, width=400)
)
if LOCAL:
    for j in ["json", "png", "svg"]:
        layer.save("visualisation/" + f + "." + j)
layer

Fontconfig error: Cannot load default config file
Fontconfig error: Cannot load default config file


alt.LayerChart(...)

# Fig 2

In [9]:
raw_file_path = "raw/Figures.xlsx"
if not LOCAL:
    raw_file_path = eco_git_path.split("/data")[0] + "/" + raw_file_path
df = (
    pd.read_excel(
        raw_file_path,
        skiprows=6,
        nrows=10,
    )
    .dropna(how="all", axis=1)
    .dropna(how="all", axis=0)
)
df = df.set_index("Category").stack().reset_index()
df.columns = ["category", "obesity", "value"]

In [10]:
f = "fig2_deprivation"
f2 = eco_git_path + f + ".csv"
if LOCAL:
    df.to_csv("data/" + f + ".csv")
    f += local_suffix
    open("visualisation/" + f + ".html", "w").write(
        vega_embed.replace(
            "JSON_PATH",
            (eco_git_path + f + ".csv")
            .replace("/data/", "/visualisation/")
            .replace(".csv", ".json"),
        )
    )
    f2 = df
df.head()

,category,obesity,value
0,1,Obese,27.458032
1,1,Severely obese,7.525181
2,2,Obese,26.560227
3,2,Severely obese,6.804313
4,3,Obese,25.106422


In [11]:
bars = (
    alt.Chart(f2)
    .mark_bar(size=14, opacity=0.9)
    .encode(
        color=alt.Color(
            "obesity:N",
            scale=alt.Scale(
                domain=df["obesity"].unique(),
                range=[colors["eco-turquiose"], colors["eco-blue"]],
            ),
        )
    )
)
text = bars.mark_text(dx=-12, dy=1, color="white").encode(
    text=alt.Text("SAmmount:Q", format=".0f"),
    color=alt.condition(
        datum.SAmmount < 3,
        alt.ColorValue(None),
        alt.ColorValue("white"),
    ),
)
labels = (
    alt.Chart(
        pd.DataFrame(
            [
                {"SAmmount": 0.5, "category": "1", "t": "most deprived"},
                {"SAmmount": 0.5, "category": "10", "t": "least deprived"},
            ]
        )
    )
    .mark_text(yOffset=1, size=10, align="left", color="white", baseline="middle")
    .encode(text="t")
)

layer = (
    (bars + text + labels)
    .encode(
        x=alt.X(
            "SAmmount:Q",
            stack="zero",
            axis=alt.Axis(
                grid=False,
                title="%",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleAnchor="end",
                titleY=-15,
                titleFontSize=10,
                titleFontWeight="normal",
            ),
            # scale=alt.Scale(domain=[0, 35]),
        ),
        y=alt.Y(
            "category:O",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="Index of Multiple deprivation",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleX=0,
                titleY=-5,
                titleAngle=0,
                titleFontSize=10,
                titleFontWeight="normal",
                titleAlign="left",
            ),
        ),
        order="category:O",
    )
    .properties(height=alt.Step(20), width=300)
    .configure_legend(title=None)
    .configure_view(stroke=None)
    .transform_aggregate(SAmmount="sum(value)", groupby=["category", "obesity"])
)
if LOCAL:
    for j in ["json", "png", "svg"]:
        layer.save("visualisation/" + f + "." + j)
layer

Fontconfig error: Cannot load default config file
Fontconfig error: Cannot load default config file


alt.LayerChart(...)

# Fig 3

In [12]:
raw_file_path = "raw/Figures.xlsx"
if not LOCAL:
    raw_file_path = eco_git_path.split("/data")[0] + "/" + raw_file_path
df = (
    pd.read_excel(
        raw_file_path,
        skiprows=19,
        nrows=14,
    )
    .dropna(how="all", axis=1)
    .dropna(how="all", axis=0)
)
df.columns = ["year", "most", "least"]
df["year"] = df["year"].str.split("/").str[0].astype(str)
df["category"] = "obese"
df2 = (
    pd.read_excel(
        "raw/Figures.xlsx",
        skiprows=35,
        nrows=14,
    )
    .dropna(how="all", axis=1)
    .dropna(how="all", axis=0)
)
df2.columns = ["year", "most", "least"]
df2["year"] = df["year"].str.split("/").str[0].astype(str)
df2["category"] = "severely"
df = pd.concat([df, df2])

In [13]:
f = "fig3_deprivation_trend"
f3 = eco_git_path + f + ".csv"
if LOCAL:
    df.to_csv("data/" + f + ".csv")
    f += local_suffix
    open("visualisation/" + f + ".html", "w").write(
        vega_embed.replace(
            "JSON_PATH",
            (eco_git_path + f + ".csv")
            .replace("/data/", "/visualisation/")
            .replace(".csv", ".json"),
        )
    )
    f3 = df
df.head()

,year,most,least,category
0,2006,21.493809,12.988846,obese
1,2007,22.459264,13.553442,obese
2,2008,22.636763,13.294991,obese
3,2009,23.461627,13.641194,obese
4,2010,23.678682,13.810939,obese


In [14]:
base = alt.Chart(f3).encode(
    x=alt.X(
        "year:T",
        axis=alt.Axis(
            grid=False,
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
        ),
    ),
    order="category:O",
)
area1 = base.mark_area(
    opacity=0.3,
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-dot"], offset=0.9),
        ],
        x1=0.8,
        x2=1,
        y1=1,
        y2=0,
    ),
).encode(alt.Y("least:Q"), alt.Y2("most:Q"))
area1b = base.mark_area(
    opacity=0.3,
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0.5),
            alt.GradientStop(color=colors["eco-turquiose"], offset=0),
        ],
        x1=1,
        x2=1,
        y1=1,
        y2=0,
    ),
).encode(
    alt.Y(
        "least:Q",
        axis=alt.Axis(
            grid=True,
            title="% of children age 10-11 that are",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            labelAlign="left",
            ticks=False,
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=20,
            titleY=-5,
            titleAngle=0,
            titleFontSize=10,
            titleFontWeight="normal",
            titleAlign="left",
            tickCount=4,
            format=".0f",
        ),
        scale=alt.Scale(domain=[0, 30]),
    ),
    alt.Y2("most:Q"),
)
area1c = base.mark_area(
    opacity=0.3,
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0.5),
            alt.GradientStop(color=colors["eco-blue"], offset=0),
        ],
        x1=1,
        x2=1,
        y1=1,
        y2=0,
    ),
).encode(
    alt.Y(
        "least:Q",
        axis=alt.Axis(
            grid=True,
            title="% of children age 10-11 that are",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            labelAlign="left",
            ticks=False,
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=15,
            titleY=-5,
            titleAngle=0,
            titleFontSize=10,
            titleFontWeight="normal",
            titleAlign="left",
            tickCount=4,
            format=".0f",
        ),
        scale=alt.Scale(domain=[0, 8]),
    ),
    alt.Y2("most:Q"),
)
line1 = base.mark_line(width=2, color=colors["eco-turquiose"]).encode(alt.Y("least:Q"))
line1b = base.mark_line(width=2, color=colors["eco-blue"]).encode(alt.Y("least:Q"))
line2 = base.mark_line(width=2, color=colors["eco-dot"]).encode(alt.Y("most:Q"))
label1 = (
    alt.Chart(
        pd.DataFrame(
            [{"x": "2013-03-15", "y": 31.9, "t": "Obese", "c": colors["eco-turquiose"]}]
        )
    )
    .mark_text(yOffset=5, size=11, align="left", fontWeight="bold")
    .encode(text="t", x="x:T", y="y:Q", color=alt.Color("c:N", scale=None))
)
label1b = (
    alt.Chart(
        pd.DataFrame(
            [
                {
                    "x": "2010-03-15",
                    "y": 11.9,
                    "t": "least dperived regions",
                    "c": colors["eco-turquiose"],
                },
                {
                    "x": "2009-03-15",
                    "y": 26.9,
                    "t": "most dperived regions",
                    "c": colors["eco-dot"],
                },
                {
                    "x": "2016-12-31",
                    "y": 29.2,
                    "t": "+13.3 pp",
                    "c": colors["eco-dot"],
                },
                {
                    "x": "2006-04-01",
                    "y": 24.8,
                    "t": "+8.5 pp",
                    "c": colors["eco-dot"],
                },
            ]
        )
    )
    .mark_text(yOffset=5, size=10, align="left")
    .encode(text="t", x="x:T", y="y:Q", color=alt.Color("c:N", scale=None))
)
label2 = (
    alt.Chart(
        pd.DataFrame(
            [
                {
                    "x": "2013-01-01",
                    "y": 8.51,
                    "t": "Severely obese",
                    "c": colors["eco-blue"],
                }
            ]
        )
    )
    .mark_text(yOffset=5, size=11, align="left", fontWeight="bold")
    .encode(text="t", x="x:T", y="y:Q", color=alt.Color("c:N", scale=None))
)
label2b = (
    alt.Chart(
        pd.DataFrame(
            [
                {
                    "x": "2010-03-15",
                    "y": 1.4,
                    "t": "least deprived regions",
                    "c": colors["eco-blue"],
                },
                {
                    "x": "2009-11-15",
                    "y": 6.7,
                    "t": "most deprived regions",
                    "c": colors["eco-dot"],
                },
                {
                    "x": "2016-12-31",
                    "y": 7.8,
                    "t": "+5.3 pp",
                    "c": colors["eco-dot"],
                },
                {
                    "x": "2006-03-01",
                    "y": 5.8,
                    "t": "+3.1 pp",
                    "c": colors["eco-dot"],
                },
            ]
        )
    )
    .mark_text(yOffset=5, size=10, align="left")
    .encode(text="t", x="x:T", y="y:Q", color=alt.Color("c:N", scale=None))
)
layer1 = (
    (area1 + area1b + line1 + line2 + label1 + label1b).properties(
        height=220, width=300
    )
).transform_filter("datum.category=='obese'")
layer2 = (
    (area1 + area1c + line1b + line2 + label2 + label2b).properties(
        height=220, width=300
    )
).transform_filter("datum.category=='severely'")
layer = (
    alt.hconcat(layer1, layer2, spacing=20)
    .configure_view(stroke=None)
    .properties(title="")
)
if LOCAL:
    for j in ["json", "png", "svg"]:
        layer.save("visualisation/" + f + "." + j)
layer

Fontconfig error: Cannot load default config file
Fontconfig error: Cannot load default config file


alt.HConcatChart(...)

# Fig 4

In [15]:
raw_file_path = "raw/Figures.xlsx"
if not LOCAL:
    raw_file_path = eco_git_path.split("/data")[0] + "/" + raw_file_path
df = (
    pd.read_excel(
        raw_file_path,
        skiprows=51,
        nrows=8,
    )
    .dropna(how="all", axis=1)
    .dropna(how="all", axis=0)
)
df.columns = ["year", "value"]
df["year"] = df["year"].str.split("/").str[0].astype(str)

In [16]:
f = "fig4_hospitalisation"
f4 = eco_git_path + f + ".csv"
if LOCAL:
    df.to_csv("data/" + f + ".csv")
    f += local_suffix
    open("visualisation/" + f + ".html", "w").write(
        vega_embed.replace(
            "JSON_PATH",
            (eco_git_path + f + ".csv")
            .replace("/data/", "/visualisation/")
            .replace(".csv", ".json"),
        )
    )
    f4 = df
df.head()

,year,value
0,2011,530.4
1,2012,568.1
2,2013,709.8
3,2014,845.9
4,2015,995.5


In [17]:
base = alt.Chart(f4).encode(
    x=alt.X(
        "year:T",
        axis=alt.Axis(
            grid=False,
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
        ),
    )
)
area1 = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.8,
    stroke=colors["eco-gray"],
    strokeWidth=0.5,
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-dot"], offset=0.9),
        ],
        x1=0.8,
        x2=1,
        y1=1,
        y2=0,
    ),
).encode(
    y=alt.Y(
        "value:Q",
        axis=alt.Axis(
            grid=True,
            title="persons / 100,000 population",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            labelAlign="left",
            ticks=False,
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=30,
            titleY=45,
            titleAngle=0,
            titleFontSize=10,
            titleFontWeight="normal",
            titleAlign="left",
            tickCount=4,
            format=".0f",
        ),
    )
)
layer = (
    (area1)
    .configure_view(stroke=None)
    .properties(title="")
    .properties(height=300, width=400)
)
if LOCAL:
    for j in ["json", "png", "svg"]:
        layer.save("visualisation/" + f + "." + j)
layer

Fontconfig error: Cannot load default config file
Fontconfig error: Cannot load default config file


alt.Chart(...)